In [1]:
from langgraph.graph import StateGraph,START,END
from typing import TypedDict
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import Literal
from typing import TypedDict, Literal, Annotated
from langchain_core.messages import SystemMessage, HumanMessage
import time

In [2]:
import pandas as pd
import operator

In [3]:
from typing import TypedDict
from typing_extensions import Annotated

class classification_state(TypedDict):
    description: str
    class_category: str
    class_category_reason: str
    classification_prompt: str
    benign_prompt: str
    pathogenic_prompt: str
    both_prompt: str
    class_truth: str
    ground_truth: list
    response: Annotated[list[str], operator.add]
    reason: str
    class_iter: int
    current_iter: int
    max_iter: int

In [25]:
model = ChatGoogleGenerativeAI(model='gemini-2.5-flash',temperature=0.3)

In [26]:
from pydantic import BaseModel, Field

class DescriptionEvaluation(BaseModel):
    evaluation: Literal['Pathogenic', 'Benign','Both'] = Field(..., description="ACMG Attribute class only accepted values are Pathogenic, Benign, Both")
    reason: str = Field(..., description="Reason for the classification.")

class ACMGEvaluation(BaseModel):
    evaluation: list[str] = Field(..., description="ACMG Attribute")
    reason: str = Field(..., description="Reason for the classification.")

class Prompt(BaseModel):
    prompt: str = Field(..., description="Attribute Prompt")

In [27]:
classification_model = model.with_structured_output(DescriptionEvaluation)
attribute_model = model.with_structured_output(ACMGEvaluation)
prompt_model = model.with_structured_output(Prompt)

In [28]:
def description_classification(state: classification_state):
    description= state['description']
    messages = [
    SystemMessage(content=
    f"""you are a ACMG Attribute classifier here is some extra details: {state['classification_prompt']}"""
     ),
    HumanMessage(content=f""" Here is the given description:
     {description}, 
        ########### respond only in the structure format:
        evaluation: description="ACMG Attribute class only accepted values are Pathogenic, Benign, Both
        feedback: Reason for the classification.
    """)
    ]
    response = classification_model.invoke(messages)
    return {"class_category":response.evaluation,"class_category_reason":response.reason}

In [29]:
def benign_classification(state: classification_state):
    description= state['description']
    messages = [
    SystemMessage(content=f"{state['benign_prompt']}"),
    HumanMessage(content=f""" Here is the given description:
     {description}, 
        ########### respond only in the structure format:
        evaluation: list[str] = Field(..., description="ACMG Attribute")
    reason: str = Field(..., description="Reason for the classification.")
    """)
    ]
    response = attribute_model.invoke(messages)
    return {"response":response.evaluation,"reason":response.reason}

In [30]:
def pathogenic_classification(state: classification_state):
    description= state['description']
    messages = [
    SystemMessage(content=f"{state['pathogenic_prompt']}"),
    HumanMessage(content=f""" Here is the given description:
     {description}, 
        ########### respond only in the structure format:
        evaluation: list[str] = Field(..., description="ACMG Attribute")
    reason: str = Field(..., description="Reason for the classification.")
    """)
    ]
    response = attribute_model.invoke(messages)
    return {"response":response.evaluation,"reason":response.reason}

In [31]:
def both_classification(state: classification_state):
    description= state['description']
    messages = [
    SystemMessage(content=f"{state['both_prompt']}"),
    HumanMessage(content=f""" Here is the given description:
     {description}, 
        ########### respond only in the structure format:
        evaluation: list[str] = Field(..., description="ACMG Attribute")
    reason: str = Field(..., description="Reason for the classification.")
    """)
    ]
    response = attribute_model.invoke(messages)
    return {"response":response.evaluation,"reason":response.reason}

In [32]:
def classification_check(state: classification_state):
    class_check = {"next":""}
    if state['class_iter']>state['max_iter']:
        class_check["next"] = "continue"
    else:
        if state['class_category'] != state['class_truth']:
            class_check["next"] = "check"
        else:
            class_check["next"] = "continue"
    return class_check

In [33]:
def response_check(state: classification_state):
    if state['current_iter'] > state['max_iter']:
        return {"next": "end"}
    else:
        for i in eval(str(state['response'])):
            if i not in [state['ground_truth']]:
                return {"next": "check"}
    return {"next": "end"}

In [34]:
def benign_prompt_update(state: classification_state):
    description= state['description']
    messages = [
    SystemMessage(content=f"You are a ACMG Attribute classifier prompt developer for the given description, your task is to update the prompt to add the additional details which will correct your llm Prediction as your current prompt is returning {state['response']} but in reality it's {state['ground_truth']}, here is the reason why your LLM gave the old response {state['reason']}, now update the system prompt to get the correct response for the upcoming description and in response only return the prompt no extra text, give in points and include the previous prompts, here is the old prompt {state['benign_prompt']} and summarize it. Dont use any markdown characters, only text, make it as detail as possible so that that prompt i can use for any other description classification also, like a generalized prompt for the classification, and make sure i will only use this for benign classification"),
    HumanMessage(content=f""" Here is the given description:{description}
    ########### respond only in the structure format:
        prompt: str = Attribute Prompt
    """)
    ]
    response = prompt_model.invoke(messages)
    itter = state['current_iter']+1
    return {"benign_prompt": response.prompt,"current_iter": itter}

In [35]:
def pathogenic_prompt_update(state: classification_state):
    description= state['description']
    messages = [
    SystemMessage(content=f"You are a ACMG Attribute classifier prompt developer for the given description, your task is to update the prompt to add the additional details which will correct your llm Prediction as your current prompt is returning {state['response']} but in reality it's {state['ground_truth']}, here is the reason why your LLM gave the old response {state['reason']}, now update the system prompt to get the correct response for the upcoming description and in response only return the prompt no extra text, give in points and include the previous prompts, here is the old prompt {state['pathogenic_prompt']} and summarize it. Dont use any markdown characters, only text,  make sure only prompt for pathogenic classification , make it as detail as possible so that that prompt i can use for any other description classification also, like a generalized prompt for the classification, and make sure i will only use this for pathogenic classification"),
    HumanMessage(content=f""" Here is the given description:{description}
    ########### respond only in the structure format:
        prompt: str = Attribute Prompt
    """)
    ]
    response = prompt_model.invoke(messages)
    itter = state['current_iter']+1
    return {"pathogenic_prompt": response.prompt,"current_iter": itter}

In [36]:
def both_prompt_update(state: classification_state):
    description= state['description']
    messages = [
    SystemMessage(content=f"You are a ACMG Attribute classifier prompt developer for the given description, your task is to update the prompt to add the additional details which will correct your llm Prediction as your current prompt is returning {state['response']} but in reality it's {state['ground_truth']}, here is the reason why your LLM gave the old response {state['reason']}, now update the system prompt to get the correct response for the upcoming description and in response only return the prompt no extra text, give in points and include the previous prompts, here is the old prompt {state['both_prompt']} and summarize it. Dont use any markdown characters, only text"),
    HumanMessage(content=f""" Here is the given description:{description}
    ########### respond only in the structure format:
        prompt: str = Attribute Prompt
    """)
    ]
    response = prompt_model.invoke(messages)
    itter = state['current_iter']+1
    return {"both_prompt": response.prompt,"current_iter": itter}

In [37]:
def prompt_update(state: classification_state):
    description= state['description']
    messages = [
    SystemMessage(content=f"You are a ACMG Attribute classifier prompt developer for the given description, your task is to update the prompt to add the additional details which will correct your llm Prediction as your current prompt is returning {state['class_category']} but in reality it's {state['class_truth']}, here is the reason why your LLM gave the old response {state['class_category_reason']}, now update the system prompt to get the correct response for the upcoming description"),
    HumanMessage(content=f""" Here is the given description:{description}
    ########### respond only in the structure format:
        prompt: str = Attribute Prompt
    """)
    ]
    response = prompt_model.invoke(messages)
    itter = state['class_iter']+1
    return {"classification_prompt": response.prompt,"class_iter": itter}

In [38]:
def workflow_selection(state: classification_state):
    workflow = {"next":""}
    if state['class_category'] == 'Benign':
        workflow['next'] = "benign_classification"
    elif state['class_category'] == 'Pathogenic':
        workflow['next'] = 'pathogenic_classification'
    elif state['class_category'] == 'Both':
        workflow['next'] = "both_classification"
    else:
        workflow['next'] = "end"
    return workflow

In [39]:
graph = StateGraph(classification_state)

graph.add_node('description_classification', description_classification)
graph.add_node('classification_check',classification_check)
graph.add_node('prompt_update',prompt_update)
graph.add_node('workflow_selection', workflow_selection)

graph.add_node('benign_classification', benign_classification)
graph.add_node('pathogenic_classification', pathogenic_classification)
graph.add_node('both_classification', both_classification)

graph.add_node('benign_response_check', response_check)
graph.add_node('pathogenic_response_check', response_check)
graph.add_node('both_response_check', response_check)

graph.add_node('benign_prompt_update', benign_prompt_update)
graph.add_node('pathogenic_prompt_update', pathogenic_prompt_update)
graph.add_node('both_prompt_update', both_prompt_update)

graph.add_edge(START, 'description_classification')
graph.add_edge('description_classification','classification_check')
graph.add_conditional_edges('classification_check',lambda x: x["next"],{"continue":"workflow_selection","check":"prompt_update"})

graph.add_edge('prompt_update','description_classification')

graph.add_conditional_edges('workflow_selection',lambda x: x["next"],
    {"end": END, "benign_classification": "benign_classification","pathogenic_classification":"pathogenic_classification","both_classification":"both_classification"})

graph.add_edge('both_classification','both_response_check')
graph.add_edge('benign_classification','benign_response_check')
graph.add_edge('pathogenic_classification','pathogenic_response_check')

graph.add_conditional_edges(
    'benign_response_check',
    lambda x: x["next"],
    {"end": END, "check": "benign_prompt_update"}
)

graph.add_conditional_edges(
    'pathogenic_response_check',
    lambda x: x["next"],
    {"end": END, "check": "pathogenic_prompt_update"}
)

graph.add_conditional_edges(
    'both_response_check',
    lambda x: x["next"],
    {"end": END, "check": "both_prompt_update"}
)


graph.add_edge('benign_prompt_update', 'benign_classification')
graph.add_edge('pathogenic_prompt_update', 'pathogenic_classification')
graph.add_edge('both_prompt_update', 'both_classification')

workflow = graph.compile()


In [40]:
df=pd.read_csv('final_classification.csv')

df_balanced = df.groupby('class', group_keys=False).apply(
    lambda x: x.sample(n=33, random_state=42)
)
df_random = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

C:\Users\rishi\AppData\Local\Temp\ipykernel_3748\2288021753.py:3: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_balanced = df.groupby('class', group_keys=False).apply(


In [41]:
with open('classification_prompt.txt','r') as file:
    classification_prompt = file.readlines()
with open('benign_prompt.txt','r') as file:
    benign_prompt = file.readlines()
with open('pathogenic_prompt.txt','r') as file:
    pathogenic_prompt = file.readlines()
with open('both_prompt.txt','r') as file:
    both_prompt = file.readlines()

In [42]:
initial_state = {'description': 'The p.V217I variant (also known as c.649G>A), located in coding exon 7 of the PTEN gene, results from a G to A substitution at nucleotide position 649. The valine at codon 217 is replaced by isoleucine, an amino acid with highly similar properties. In a massively parallel functional assay using a humanized yeast model, lipid phosphatase activity for this variant was functionally neutral (Mighell TL et al. Am J Hum Genet, 2018 May;102:943-955). This variant demonstrated wild type-like intracellular protein abundance in a massively parallel functional assay (Matreyek KA et al. Nat Genet, 2018 Jun;50:874-882). This amino acid position is highly conserved in available vertebrate species. In addition, this alteration is predicted to be tolerated by in silico analysis. Since supporting evidence is limited at this time, the clinical significance of this alteration remains unclear.',
 'current_iter': 0,
 'max_iter': 3,
 'ground_truth': "['BS3']",
 'benign_prompt': benign_prompt,
 'pathogenic_prompt': pathogenic_prompt,
 'both_prompt': both_prompt,
 'initial_state': 0,
 'class_iter': 0,
 'classification_prompt': classification_prompt,
 'class_truth': 'Benign'}


In [43]:

count = 51
for i in df_random.values[52:]:
    try:
        count+=1
        print(f"Itter: {count}")
        res=workflow.invoke(initial_state)
        initial_state['initial_state'] = 0
        initial_state['class_iter'] = 0
        initial_state['ground_truth'] = i[3]
        initial_state['description'] = i[2]
        initial_state['benign_prompt'] = res['benign_prompt']
        initial_state['pathogenic_prompt'] = res['pathogenic_prompt']
        initial_state['both_prompt'] = res['both_prompt']
        initial_state['classification_prompt']=res['classification_prompt']
        initial_state['class_truth']=i[4]
        with open('benign_prompt.txt','w') as file:
            file.write(initial_state['benign_prompt'])
        with open('pathogenic_prompt.txt','w') as file:
            file.write(initial_state['pathogenic_prompt'])
        with open('both_prompt.txt','w') as file:
            file.write(initial_state['both_prompt'])
        with open('classification_prompt.txt','w') as file:
            file.write(initial_state['classification_prompt'])
        if count%10==0:
            time.sleep(10)
    except Exception as e:
        break

Itter: 52


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 250
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 20
}
].


In [ ]:
initial_state

In [ ]:
df_balanced.random()